# **Template OP on salish**

In [1]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels_beaching import DeleteParticle, Buoyancy, AdvectionRK4_3D, Stokes_drift, Beaching
from OP_functions_beaching import *

In [2]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

In [6]:
Dat=xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
path_NEMO = make_prefix(datetime(2018, 12, 23), paths['NEMO'])
Dat0=xr.open_dataset(path_NEMO + '_grid_W.nc')
Dat=xr.open_dataset(path_NEMO + '_grid_T.nc')
coord=xr.open_dataset(paths['coords'],decode_times=False)
WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
batt=xr.open_dataset(paths['mask'],decode_times=False)

## Define and save mask for distance to coast

In [6]:
# def maskcoast(ilat,jlon):
#     baty = batt.mbathy[0,:,:]
#     maski = 0
#     if baty[ilat,jlon]>1e-1:
#         if baty[ilat+1,jlon] == 0 or baty[ilat+1,jlon+1] == 0 or baty[ilat+1,jlon-1] == 0:
#             maski = 1
#         elif baty[ilat,jlon] == 0 or baty[ilat,jlon+1] == 0 or baty[ilat,jlon-1] == 0:
#             maski = 1
#         elif baty[ilat-1,jlon] == 0 or baty[ilat-1,jlon+1] == 0 or baty[ilat-1,jlon-1] == 0:
#             maski = 1
#     return maski
# maskd = np.zeros(batt.mbathy.shape)
# baty = batt.mbathy[0,:,:]
# for i in range(baty.shape[0]):
#     for j in range(baty.shape[1]):
#         maskd[0,i,j]=maskcoast(i,j)
# batt=xr.open_dataset(paths['mask'],decode_times=False)
# dist=xr.DataArray(attrs={'d':maskd})
# batt['d']=(batt.mbathy.dims,maskd)
# batt.to_netcdf(path='/ocean/jvalenti/MOAD/grid/mesh_maskd201702.nc')

In [6]:
#clat,clon = p_unidist(coord.gphif[0,:,:],coord.glamf[0,:,:],batt.mbathy[0,:,:],10,10)
with open('clat.txt') as f:
    clat = f.read()
    clat= clat[1:-1]
    clat0 = clat.split(",")
    f.close()
with open('clon.txt') as f:
    clon = f.read()
    clon=clon[1:-1]
    clon0 = clon.split(",")
    f.close()
    
clat,clon=[],[]
for i in range(len(clat0)):
    clat.append(float(clat0[i]))
    clon.append(float(clon0[i]))

## Definitions

In [7]:
start = datetime(2018, 10, 1) #Start date
# Set Time length [days] and timestep [seconds]
length = 1 
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = len(clat) # number of deploying locations
n = 1 # 1000   # number of particles per location
dmin = list(np.zeros(len(clat))) #minimum depth
dd = 5 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [8]:
from parcels import Variable

class MPParticle(JITParticle):        
    ro = Variable('ro', initial = 1380)           
    diameter = Variable('diameter', initial = 1.6e-5)
    length = Variable('length', initial = 61e-5)
    sediment = Variable('sediment', initial = 0)
    beached = Variable('beached', initial = 0)


In [9]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [10]:
#Set start date time and the name of the output file
name = 'beaching' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/results/beaching20181001_1n_20181002_1n.nc


## Simulation

In [10]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

filenames,variables,dimensions=filename_set(start,length,['Bathy'],local)
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

         It will be opened with no decoding. Filling values might be wrongly parsed.


In [11]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

k_sink = pset.Kernel(Buoyancy)
k_waves = pset.Kernel(Stokes_drift)

pset.execute(AdvectionRK4_3D + k_sink + k_waves,
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DBuoyancyStokes_drift ==> /tmp/parcels-2894/lib443b8b580526f1070bfe75b43091bd6f_0.so


Particle 3324 lost !! [5400.0, 3.1665896505419564, 50.445370529070736, -126.01713916776767]
Particle 3338 lost !! [5400.0, 4.77081063736257, 50.4473312523828, -126.06284156751235]
Particle 3339 lost !! [5400.0, 2.3008386812720545, 50.45873102462346, -126.0254409348264]
Particle 3340 lost !! [5400.0, 3.7368314604624295, 50.46575668473586, -126.00059523752915]
Particle 3347 lost !! [5400.0, 0.6124347532022645, 50.459751532340206, -126.09993151903237]
Particle 3348 lost !! [5400.0, 0.1483663196828655, 50.465996407585266, -126.07564544452435]
Particle 3355 lost !! [5400.0, 4.807910152410411, 50.47519751980796, -126.13810495752902]
Particle 3356 lost !! [5400.0, 2.4004030096966256, 50.47846953342501, -126.11379622760758]
Particle 3325 lost !! [14760.0, 5.01663133380272, 50.44140428849729, -125.99974468398645]


INFO: Temporary output files are stored in /home/jvalenti/MOAD/results/out-NGIJWTNF.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/results/out-NGIJWTNF" to convert these to a NetCDF file during the run.
 18% (16200.0 of 86400.0) |##            | Elapsed Time: 0:00:02 ETA:   0:01:24

Particle 3313 lost !! [23220.0, 62.16715870877235, 50.43645320179828, -125.99939286108611]


 20% (18000.0 of 86400.0) |##            | Elapsed Time: 0:00:03 ETA:   0:00:56

...

 35% (30600.0 of 86400.0) |####          | Elapsed Time: 0:00:15 ETA:   0:01:20

Particle 3276 lost !! [36360.0, 9.5460723368446, 50.434240948643684, -125.99969216784476]


 95% (82800.0 of 86400.0) |############# | Elapsed Time: 0:00:49 ETA:   0:00:01

Particle 3327 lost !! [87480.0, 11.952174989132324, 50.430866054588606, -125.99988135213009]


100% (86400.0 of 86400.0) |##############| Elapsed Time: 0:00:50 Time:  0:00:50
/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)
